# Big Data Modeling and Management Assigment


## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating distinct types of data. For this first project we will be taking a graph database to analyse beer and breweries!   

_For reference the dataset used for this project has been extracted from [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. Even though the author does not present metada on the origin of the data it is probably a collection of open data from places like [beeradvocate](https://www.beeradvocate.com/)_ 

#### Problem description

Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions. Submit the results by following the instructions

#### Connection details to the neo4j database
```
Host: rhea.isegi.unl.pt:7474  
Username: neo4j  
Password: F3cfcrnvBev57KZ8mcMk78L9wHgJVZuJ 
Connect URL : bolt://rhea.isegi.unl.pt:7687
```


#### Questions


0. __Example Question__ _How many beers does the database contain?_
1. How many different countries exist in the database?
1. Most reviews:  
    1. Which `Beer` has the most reviews?  
    1. Which `Brewery` has the most reviews for its beers?
    1. Which `Country` has the most reviews for its beers? 
1. Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?
1. Which Portuguese brewery has the most beers?
1. From those beers (the ones returned from the previous question), which has the most reviews?
1. On average how many different beer styles does each brewery produce?
1. Which brewery produces the strongest beers according to ABV?
1. If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?
1. Using Graph Algorithms answer **two** of the following questions:
    1. Which two Countries are most similiar when it comes to their **top 10** most produced Beer styles?
    1. Which beer has the most similar reviews as the beer `Super Bock Stout`?
    1. Which user is the most influential when it comes to reviews made?
    1. Which beer styles are more central when it comes the amount of beers? \
    Note: In case of a tie for the top entity, in terms of metrics outputed from the algorithms, **simply output the first.**
1. If you had to pick 3 beers to recommend using only this database, which would you pick and why?



 Questions 8 to 10 are somewhat open, which means we'll also be evaluating the reasoning behind your answer. So there aren't necessarily bad results there are only wrong criteria, explanations or execution. 
 
##### Groups  

Groups should have 4 to 5 people  
You should register your group on moodle. An email will be going out to everyone with the credentials for the database to use when storing the results.


##### Submission      

Submission of the query results to be done to the group's redis database (explained on the first class, credentials sent via email).  
The following format is expected:
```
    >>> redis.set("0", "358873")
```

This result should be the anwser of the group to question 0

The code used to produce the results and respective explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of May 2nd**

##### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and simplicity of the solution.  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

**Note:**
Remember the Neo4j is a shared database and when creating in-memory graphs please use your group's prefix.  
Ex. Instead of `my-graph` as the name of your graph please use `group0-my-graph`.

In [1]:
import py2neo

username="neo4j"
password="F3cfcrnvBev57KZ8mcMk78L9wHgJVZuJ"
host="rhea.isegi.unl.pt"
port="7474"

group22_graph = py2neo.Graph(f"http://{username}:{password}@{host}:{port}")
group22_graph.run("MATCH (u) RETURN count(*)").data()

[{'count(*)': 9647598}]

In [2]:
group22_graph.run(
    """
      CALL db.labels()
    """
).data()

[{'label': 'Breweries'},
 {'label': 'Country'},
 {'label': 'Beers'},
 {'label': 'Reviews'},
 {'label': 'Username'},
 {'label': 'Style'},
 {'label': 'TODELETE'}]

In [3]:
group22_graph.run(
    """
      CALL db.relationshipTypes()
    """
).data()

[{'relationshipType': 'FROM'},
 {'relationshipType': 'BREWED_AT'},
 {'relationshipType': 'MADE'},
 {'relationshipType': 'ABOUT'},
 {'relationshipType': 'OF_TYPE'}]

## Question 1

How many different countries exist in the database?


In [4]:
result1 = group22_graph.run("""
        MATCH (c:Country)
        RETURN count(c) as NumOfCountries
""").data()
result1

[{'NumOfCountries': 200}]

## Question 2

Most reviews:

2.A. Which Beer has the most reviews?

In [5]:
# 2.A. 
result2a = group22_graph.run("""
        MATCH (:Reviews)-[]-(b:Beers) //there is only one relationship between these node labels so its ok to leave it undefined//
        RETURN b.name as Beer, COUNT(*) as NumOfReviews
        ORDER BY NumOfReviews DESC 
        LIMIT 1
""").data()
result2a

[{'Beer': 'IPA', 'NumOfReviews': 31387}]

2.B. Which Brewery has the most reviews for its beers?

In [16]:
# 2.B.
result2b = group22_graph.run("""
        MATCH (:Reviews)-[]-(:Beers)-[]-(br:Breweries)
        RETURN br.name as Brewery, COUNT(*) as NumOfReviews
        ORDER BY NumOfReviews DESC 
        LIMIT 1
""").data()
result2b

[{'Brewery': 'Sierra Nevada Brewing Co.', 'NumOfReviews': 175161}]

2.C Which Country has the most reviews for its beers?

In [14]:
# 2.C
result2c = group22_graph.run("""
        MATCH (:Reviews)-[]-(:Beers)-[]-(br:Breweries)-[]-(c:Country)
        RETURN c.name as Country, COUNT(*) as NumOfReviews
        ORDER BY NumOfReviews DESC 
        LIMIT 1
""").data()
result2c

[{'Country': None, 'NumOfReviews': 9072914}]

## Question 3

Find the user/users that have the most shared reviews (reviews of the same beers) with the user CTJman?

In [2]:
result3 = group22_graph.run("""
        MATCH (u:Username)-[]-(:Reviews)-[]-(b:Beers)<-[]-(:Reviews)<-[]-(uCTJman:Username{user_name:'CTJman'})
        RETURN u.user_name, count(*) as NumOfReviews
        ORDER BY NumOfReviews DESC 
        LIMIT 1
""").data()
result3

[{'u.user_name': 'acurtis', 'NumOfReviews': 1428}]

## Question 4

Which Portuguese brewery has the most beers?

In [17]:
result4 = group22_graph.run("""
        MATCH (:Beers)-[]-(bw:Breweries)-[]-(:Country{country_digit:'PT'})
        RETURN bw.name as Brewery, count(*) as NumOfBeers
        ORDER BY NumOfBeers DESC
        LIMIT 1
""").data()
result4   

[{'Brewery': 'Dois Corvos Cervejeira', 'NumOfBeers': 40}]

## Question 5

From those beers (the ones returned from the previous question), which has the most reviews?

In [19]:
result5 = group22_graph.run("""
        MATCH (:Username)-[]-(:Reviews)-[]-(b:Beers)-[]-(:Breweries{name:'Dois Corvos Cervejeira'})
        RETURN b.name as Beer, count(*) as NumOfReviews
        ORDER BY NumOfReviews DESC
        LIMIT 1
""").data()
result5

[{'Beer': 'Finisterra', 'NumOfReviews': 10}]

## Question 6
On average how many different beer styles does each brewery produce?

In [6]:
result6 = group22_graph.run(f"""
    MATCH (s:Style)-[]-(:Beers)-[]-(bw:Breweries)
    WITH bw.name as Brewery, count(distinct(s)) as numdiffstyle
    RETURN avg(numdiffstyle) as AVGStylePerBrewery
""").data()
result6

[{'AVGStylePerBrewery': 10.669977315921736}]

## Question 7
Which brewery produces the strongest beers according to ABV?


In [32]:
result7 = group22_graph.run(f"""
    MATCH (b:Beers)-[]-(bw:Breweries)
    WHERE b.abv <> 'Unknown'
    RETURN bw.name as Brewery, avg(toFloat(b.abv)) as MeanABV
    ORDER BY MeanABV DESC
    LIMIT 1 
""").data()
result7

[{'Brewery': '1648 Brewing Company Ltd', 'MeanABV': 25.57777777777778}]

## Question 8
If I typically enjoy a beer due to its aroma and appearance, which beer style should I try?

In [ ]:
# Look for beers with taste and look equal to 5 
result8 = group22_graph.run("""
    MATCH (r:Reviews)-[]-(:Beers)-[]-(s:Style)
    WHERE r.taste='5' and r.look='5'
    RETURN s.name, r.taste, r.look, count(*) as NumOfReviews
    ORDER BY NumOfReviews DESC
    LIMIT 1
""").data()
result8

## Question 9
Using Graph Algorithms answer the questions: A and D

9.A.Which Countries are most similiar when it comes to the most produced Beer styles

In [8]:
# Confirm if graph already exists
graph_exists = group22_graph.run("""
        CALL gds.graph.exists('beer-graphs-022-9a')
""").data()
graph_exists

[{'graphName': 'beer-graphs-022-9a', 'exists': False}]

In [9]:
# Cypher Projection - Amount of produced Beers Styles by Country
graph9a = group22_graph.run("""
    CALL gds.graph.create.cypher(
    'beer-graphs-022-9a',
    'MATCH (c:Country) RETURN id(c) as id UNION ALL MATCH (s:Style) RETURN id(s) as id',
    'MATCH (c:Country)<-[]-(bw1:Breweries)<-[]-(b1:Beers)-[]->(s:Style) WITH c as country, s as style, count(*) as timesforstyle ORDER BY timesforstyle DESC WITH country, collect(style)[..10] as top_10 UNWIND top_10 as u_top_10 RETURN id(country) AS source, id(u_top_10) AS target')
""").data()
graph9a

In [10]:
# Stream results of the similarity algorithm
result9a = group22_graph.run("""
        CALL gds.nodeSimilarity.stream('beer-graphs-022-9a')
        YIELD node1, node2, similarity
        RETURN gds.util.asNode(node1).country_digit AS Country1, gds.util.asNode(node2).country_digit AS Country2, similarity
        ORDER BY similarity DESC
        LIMIT 10
""").data()
result9a

[{'Country1': 'TD', 'Country2': 'TJ', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'CF', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'ST', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'YT', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'BF', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'AO', 'similarity': 1.0},
 {'Country1': 'IR', 'Country2': 'LY', 'similarity': 1.0},
 {'Country1': 'IR', 'Country2': 'BW', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'SN', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'MO', 'similarity': 1.0}]

9.D. Which beer styles are more central when it comes the amount of beers?

In [8]:
# Confirm if graph already exists
graph_exists = group22_graph.run("""
        CALL gds.graph.exists('beerstyles-graphs-022-9d')
""").data()
graph_exists

[{'graphName': 'beerstyles-graphs-022-9d', 'exists': False}]

In [ ]:
# Cypher Projection - Amount of beers by Beer Style
graph9d = group22_graph.run("""
    CALL gds.graph.create.cypher(
    'beerstyles-graphs-022-9d',
    'MATCH (bs:Style) RETURN id(bs) as id UNION ALL MATCH (b:Beers) RETURN id(b) as id',
    'MATCH (bs:Style)<-[]-(b:Beers) WITH bs as style, b as beer, count(*) as timesforstyle ORDER BY timesforstyle DESC WITH style, collect(beer)[..10] as top_10 UNWIND top_10 as u_top_10 RETURN id(style) AS source, id(u_top_10) AS target')
""").data()
graph9d

In [10]:
# Stream results of the centrality algorithm using PageRank
result9d = group22_graph.run("""
        CALL gds.pageRank.stream('beerstyles-graphs-022-9d')
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS Beer_Style, score
        ORDER BY score DESC
        LIMIT 3
""").data()
result9d

[{'Country1': 'TD', 'Country2': 'TJ', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'CF', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'ST', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'YT', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'BF', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'AO', 'similarity': 1.0},
 {'Country1': 'IR', 'Country2': 'LY', 'similarity': 1.0},
 {'Country1': 'IR', 'Country2': 'BW', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'SN', 'similarity': 1.0},
 {'Country1': 'TD', 'Country2': 'MO', 'similarity': 1.0}]

## Question 10
If you had to pick 3 beers to recommend using only this database, which would you pick and why?

In [13]:
# 3 recommended beers with the highest values in these set of properties related to the opinion of the customer regarding the beer
result10 = group22_graph.run("""
        MATCH (r:Reviews)-[]-(b:Beers)
        WHERE (r.feel='5' AND r.look='5' AND r.overall='5' AND r.smell='5' AND r.taste='5' AND r.score='5')
        RETURN b.id as BeerID, b.name as BeerName, count(*) as NumOfReviews
        ORDER BY NumOfReviews DESC
        LIMIT 3
""").data()
result10

[{'BeerID': 16814, 'BeerName': 'Heady Topper', 'NumReviews': 1024},
 {'BeerID': 7971, 'BeerName': 'Pliny The Elder', 'NumReviews': 727},
 {'BeerID': 19960,
  'BeerName': 'KBS (Kentucky Breakfast Stout)',
  'NumReviews': 569}]